Before running the file Upload all your data set on your goole drive in a zip format

In [53]:
#Mount our google drive
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [54]:
!pip3 install google-api-python-client google-auth-httplib2 google-auth-oauthlib

In [55]:
from googleapiclient.discovery import build
from googleapiclient.http import MediaIoBaseDownload
from google.oauth2.service_account import Credentials
import os
import io

# Replace 'your-service-account.json' with the path to your JSON file
SERVICE_ACCOUNT_FILE = './core-outrider-443605-q5-4a88c6e9a4e1.json'
SCOPES = ['https://www.googleapis.com/auth/drive']

# Authenticate and initialize the Drive API
creds = Credentials.from_service_account_file(SERVICE_ACCOUNT_FILE, scopes=SCOPES)
drive_service = build('drive', 'v3', credentials=creds)

# Function to list and download all files from a Google Drive folder
def download_all_files(folder_id, destination_folder='./Testdata'):
    # Ensure the destination folder exists
    if not os.path.exists(destination_folder):
        os.makedirs(destination_folder)

    # Query to list files in the specified folder
    query = f"'{folder_id}' in parents and trashed=false"
    response = drive_service.files().list(q=query, fields="files(id, name)").execute()
    files = response.get('files', [])

    if not files:
        print("No files found.")
        return

    for file in files:
        file_id = file['id']
        file_name = file['name']
        print(f"Downloading {file_name}...")

        # Request to download the file
        request = drive_service.files().get_media(fileId=file_id)
        file_path = os.path.join(destination_folder, file_name)

        # Save the file to the destination folder
        with io.FileIO(file_path, 'wb') as file:
            downloader = MediaIoBaseDownload(file, request)
            done = False
            while not done:
                status, done = downloader.next_chunk()
                print(f"Download progress: {int(status.progress() * 100)}%")
    print("All files downloaded.")

# Call the function with your folder ID
if __name__ == '__main__':
    FOLDER_ID = '1wN3ZOd0WihthEeH__Lmj_ENhoXJN6U11'  # Replace with the ID of your Google Drive folder
    download_all_files(FOLDER_ID)


Download progress: 100%
Download progress: 100%
Download progress: 100%
Download progress: 100%
Download progress: 100%
Download progress: 100%
Download progress: 100%
Download progress: 100%
Download progress: 100%
Download progress: 100%
Download progress: 100%
Download progress: 100%
Download progress: 100%
Download progress: 100%
Download progress: 100%
Download progress: 100%
Download progress: 100%
Download progress: 100%
Download progress: 100%
Download progress: 100%
Download progress: 100%
Download progress: 100%
Download progress: 100%
Download progress: 100%
Download progress: 100%
Download progress: 100%
Download progress: 100%
Download progress: 100%
Download progress: 100%
Download progress: 100%
Download progress: 100%
Download progress: 100%
Download progress: 100%
Download progress: 100%
Download progress: 100%
Download progress: 100%
Download progress: 100%
Download progress: 100%
Download progress: 100%
Download progress: 100%
Download progress: 100%
Download progres

In [63]:
#To get the average frame count
import json
import glob
import numpy as np
import cv2
import copy
#change the path accordingly
video_files =  glob.glob('./Testdata/*.mp4')
#video_files1 =  glob.glob('/content/dfdc_train_part_0/*.mp4')
#video_files += video_files1
frame_count = []
for video_file in video_files:
  cap = cv2.VideoCapture(video_file)
  frame_count.append(int(cap.get(cv2.CAP_PROP_FRAME_COUNT)))
print("frames" , frame_count)
print("Total number of videos: " , len(frame_count))
print('Average frame per video:',np.mean(frame_count))

frames [148, 148, 148, 148, 148, 148, 148, 148, 148, 148, 148, 148, 148, 148, 148, 148, 148, 148, 148, 148, 148, 148, 148, 148, 148, 148, 148, 143, 148, 148, 148, 148, 148, 148, 148, 148, 148, 148, 148, 148, 148, 148, 148, 148, 148, 148, 148, 148, 148, 148, 148, 148, 148, 148, 148, 148, 148, 148, 148, 148, 148, 148, 148, 148, 148, 148, 148, 148, 148, 148, 148, 148, 148, 148, 148, 148, 148, 146, 148, 148, 148, 148, 148, 139, 148, 148, 148, 148, 148, 148, 148, 148, 148, 148, 148, 148, 148, 148, 148, 148]
Total number of videos:  100
Average frame per video: 147.84


In [64]:
# to extract frame
def frame_extract(path):
  vidObj = cv2.VideoCapture(path)
  success = 1
  while success:
      success, image = vidObj.read()
      if success:
          yield image
!pip3 install face_recognition
!mkdir '/content/drive/My Drive/FF_REAL_Face_only_data'
import torch
import torchvision
from torchvision import transforms
from torch.utils.data import DataLoader
from torch.utils.data.dataset import Dataset
import os
import numpy as np
import cv2
import matplotlib.pyplot as plt
import face_recognition
from tqdm.autonotebook import tqdm
# process the frames
def create_face_videos(path_list,out_dir):
  already_present_count =  glob.glob(out_dir+'*.mp4')
  print("No of videos already present " , len(already_present_count))
  for path in tqdm(path_list):
    out_path = os.path.join(out_dir,path.split('/')[-1])
    file_exists = glob.glob(out_path)
    if(len(file_exists) != 0):
      print("File Already exists: " , out_path)
      continue
    frames = []
    flag = 0
    face_all = []
    frames1 = []
    out = cv2.VideoWriter(out_path,cv2.VideoWriter_fourcc('M','J','P','G'), 30, (112,112))
    for idx,frame in enumerate(frame_extract(path)):
      #if(idx % 3 == 0):
      if(idx <= 150):
        frames.append(frame)
        if(len(frames) == 4):
          faces = face_recognition.batch_face_locations(frames)
          for i,face in enumerate(faces):
            if(len(face) != 0):
              top,right,bottom,left = face[0]
            try:
              out.write(cv2.resize(frames[i][top:bottom,left:right,:],(112,112)))
            except:
              pass
          frames = []
    try:
      del top,right,bottom,left
    except:
      pass
    out.release()

mkdir: cannot create directory ‘/content/drive/My Drive/FF_REAL_Face_only_data’: File exists


In [65]:
create_face_videos(video_files,'/content/drive/My Drive/FF_REAL_Face_only_data/')

No of videos already present  50


  0%|          | 0/100 [00:00<?, ?it/s]

File Already exists:  /content/drive/My Drive/FF_REAL_Face_only_data/ofpugszjdt.mp4
File Already exists:  /content/drive/My Drive/FF_REAL_Face_only_data/spgzovqysj.mp4
File Already exists:  /content/drive/My Drive/FF_REAL_Face_only_data/caaackoecr.mp4
File Already exists:  /content/drive/My Drive/FF_REAL_Face_only_data/dinukxjoce.mp4
File Already exists:  /content/drive/My Drive/FF_REAL_Face_only_data/oznyyhvzxk.mp4
File Already exists:  /content/drive/My Drive/FF_REAL_Face_only_data/ardijxbxdm.mp4
File Already exists:  /content/drive/My Drive/FF_REAL_Face_only_data/arsayfybpz.mp4
File Already exists:  /content/drive/My Drive/FF_REAL_Face_only_data/ibkgonjbjz.mp4
File Already exists:  /content/drive/My Drive/FF_REAL_Face_only_data/oaguiggjyv.mp4
File Already exists:  /content/drive/My Drive/FF_REAL_Face_only_data/tyojwenpaf.mp4
File Already exists:  /content/drive/My Drive/FF_REAL_Face_only_data/ywvbdslsgu.mp4
File Already exists:  /content/drive/My Drive/FF_REAL_Face_only_data/dxlpzbi